
# Pytorch + HuggingFace 
# KcBert Model
**KcELECTRA**-Base FineTune with PyTorch-Lightning1.3 사용 <br>
https://www.kaggle.com/code/junbumlee/kcelectra-base-finetune-with-pytorch-lightning1-3/notebook

## Datasets
- 지역 20곳 & 각 지역별 식당 40여개 리뷰 전체 크롤링 <br>
- 강릉, 건대, 경주, 광안리, 남포동, 부평, 사당, 서귀포, 서면, 속초, 송도, 연신내, 왕십리, 일산, 전주한옥마을, 제주, 판교, 해운대, 혜화, 홍대

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
- GPU로 돌려주세요

#Preparation

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 설치
!pip install transformers # HuggingFace transformers
!pip install pandas tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 8.2 MB/s 
     |████████████████████████████████| 182 kB 58.8 MB/s 
     |████████████████████████████████| 7.6 MB 69.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AdamW, TextClassificationPipeline, BertForSequenceClassification
from tqdm.notebook import tqdm

In [ ]:
# 데이터셋 불러오기
train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')
#test_pd = pd.read_csv('/content/drive/MyDrive/test_set (1).csv')

In [ ]:
train=train.rename(columns={'평점':'score'})

In [ ]:
train["라벨링"] = train.score.map({0.5:0,1:1,1.5:2,2:3,2.5:4,3:5,3.5:6,4:7,4.5:8,5:9})
train

,리뷰,score,라벨링
0,국물이 냄새도없고 담백하고 맛있었어요,3.5,6.0
1,뷰 맛집~ \n근데 파스타는 별로에요.,3.0,5.0
2,옛날맛에서 변했어요 감튀도 냉동싸구려고 햄버거 내용물이 많이 짜졌어요,1.5,2.0
3,생각했던 스페인요리와는 조금 다르네요. 그래도 직원분들이 친절하십니다.,3.0,5.0
4,마이크로 웨이팅 번호 부르고 어디 앉아라 계속 말하는통에 음식이 입으로 들어가는지 ...,2.0,3.0
...,...,...,...
30249,배터지게 먹고 갑니다!,5.0,9.0
30250,항상 너무 친절하고 맛있는 다이닝룸이에요!!,5.0,9.0
30251,친절하고 맛있었어요,4.0,7.0
30252,맛은있는데 가격대비 양이 너믄 작아요,4.0,7.0


In [ ]:
test=test.rename(columns={'평점':'grade'})

In [ ]:
test["라벨링"] = test.grade.map({0.5:0,1:1,1.5:2,2:3,2.5:4,3:5,3.5:6,4:7,4.5:8,5:9})
test

,리뷰,grade,라벨링
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,9
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,9
2,좋아요,5.0,9
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,9
4,귀엽다,3.0,5
...,...,...,...
27924,생각보다 그냥 그래욤,3.5,6
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,9
27926,친절해요,5.0,9
27927,좋아요,5.0,9


In [ ]:
test = test.applymap(str)

In [ ]:
train = train.applymap(str)

In [ ]:
train.to_csv('train_set.csv')

In [ ]:
test.to_csv('test_set.csv')

In [ ]:
train= './train_set.csv' 
test = './test_set.csv'

In [ ]:
# GPU 사용
device = torch.device("cuda")

# Dataset

In [ ]:
class ClassifyDataset(Dataset):
  
  def __init__(self, csv_file):
    # csv 읽어오기 & NaN 처리
    self.dataset = pd.read_csv(csv_file).dropna(axis=0) 
    # 중복 제거
    self.dataset.drop_duplicates(subset=['리뷰'], inplace=True)
    # pretrained model
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:4].values # idx 행과 0,1,2 columns
    text = row[1]
    y = row[3]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt', # return pytorch tensors
        truncation=True, # reducing long sequences
        max_length=256,
        pad_to_max_length=True, # padding
        add_special_tokens=True # 문장 앞뒤로 special tocken - padding 부착
        )
    
    input_ids = inputs['input_ids'][0] 
    attention_mask = inputs['attention_mask'][0] # the position of the padded indices so that the model does not attend to them

    return input_ids, attention_mask, y

In [ ]:
train_dataset = ClassifyDataset(train)
test_dataset = ClassifyDataset(test)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

         Unnamed: 0         score           라벨링
count  24689.000000  24689.000000  24689.000000
mean   14981.643525      3.717425      6.434850
std     8770.486945      1.400143      2.800286
min        0.000000      0.500000      0.000000
25%     7327.000000      3.000000      5.000000
50%    14987.000000      4.000000      7.000000
75%    22510.000000      5.000000      9.000000
max    30253.000000      5.000000      9.000000
         Unnamed: 0         grade           라벨링
count  22951.000000  22951.000000  22951.000000
mean   13748.983138      4.484619      7.969239
std     8091.051841      0.939953      1.879905
min        0.000000      0.500000      0.000000
25%     6704.000000      4.000000      7.000000
50%    13635.000000      5.000000      9.000000
75%    20740.500000      5.000000      9.000000
max    27928.000000      5.000000      9.000000


# Create Model & Train

In [ ]:
model = BertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=10).to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model,device=0) # gpu로 변환

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [ ]:
epochs = 10
batch_size = 32

optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []
accuracies = []

for i in range(epochs): # epoch 5
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train() # forward

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader): # tqdm 진행상황 확인 
    optimizer.zero_grad()
  
    y_batch = y_batch.type(torch.LongTensor)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0] 
    loss = F.cross_entropy(y_pred, y_batch)

    loss.backward()
    optimizer.step() 

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total) 

  0%|          | 0/772 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 184.99121987819672 Accuracy: tensor(0.3969, device='cuda:0')
Batch Loss: 338.8773507475853 Accuracy: tensor(0.4477, device='cuda:0')
Batch Loss: 482.44704645872116 Accuracy: tensor(0.4694, device='cuda:0')
Batch Loss: 621.7693776488304 Accuracy: tensor(0.4839, device='cuda:0')
Batch Loss: 756.4216120243073 Accuracy: tensor(0.4947, device='cuda:0')
Batch Loss: 893.1546654701233 Accuracy: tensor(0.4985, device='cuda:0')
Batch Loss: 1023.8935682177544 Accuracy: tensor(0.5044, device='cuda:0')
Train Loss: 1119.791349887848 Accuracy: tensor(0.5065, device='cuda:0')


  0%|          | 0/772 [00:00<?, ?it/s]

Batch Loss: 126.0017284154892 Accuracy: tensor(0.5572, device='cuda:0')
Batch Loss: 254.4608542919159 Accuracy: tensor(0.5530, device='cuda:0')
Batch Loss: 382.06759107112885 Accuracy: tensor(0.5501, device='cuda:0')
Batch Loss: 508.463740170002 Accuracy: tensor(0.5511, device='cuda:0')
Batch Loss: 632.6179320216179 Accuracy: tensor(0.5534, device='cuda:0')
Batch Loss: 759.0213459730148 Accuracy: tensor(0.5532, device='cuda:0')
Batch Loss: 886.6812686920166 Accuracy: tensor(0.5516, device='cuda:0')
Train Loss: 979.4726040363312 Accuracy: tensor(0.5504, device='cuda:0')


...

Batch Loss: 88.73719191551208 Accuracy: tensor(0.6700, device='cuda:0')
Batch Loss: 171.31817144155502 Accuracy: tensor(0.6833, device='cuda:0')
Batch Loss: 256.31886184215546 Accuracy: tensor(0.6859, device='cuda:0')
Batch Loss: 341.8758264183998 Accuracy: tensor(0.6854, device='cuda:0')
Batch Loss: 427.989750623703 Accuracy: tensor(0.6855, device='cuda:0')
Batch Loss: 513.1173269152641 Accuracy: tensor(0.6853, device='cuda:0')
Batch Loss: 597.7677713036537 Accuracy: tensor(0.6852, device='cuda:0')
Train Loss: 658.5395728051662 Accuracy: tensor(0.6858, device='cuda:0')


  0%|          | 0/772 [00:00<?, ?it/s]

Batch Loss: 77.82597014307976 Accuracy: tensor(0.7259, device='cuda:0')
Batch Loss: 153.2067615389824 Accuracy: tensor(0.7278, device='cuda:0')
Batch Loss: 232.39036691188812 Accuracy: tensor(0.7216, device='cuda:0')
Batch Loss: 309.9076789319515 Accuracy: tensor(0.7195, device='cuda:0')
Batch Loss: 388.11799052357674 Accuracy: tensor(0.7194, device='cuda:0')
Batch Loss: 463.50491774082184 Accuracy: tensor(0.7213, device='cuda:0')
Batch Loss: 543.1065472066402 Accuracy: tensor(0.7192, device='cuda:0')
Train Loss: 599.2396767437458 Accuracy: tensor(0.7199, device='cuda:0')


  0%|          | 0/772 [00:00<?, ?it/s]

Batch Loss: 67.43173086643219 Accuracy: tensor(0.7619, device='cuda:0')
Batch Loss: 135.91390934586525 Accuracy: tensor(0.7589, device='cuda:0')
Batch Loss: 207.223792552948 Accuracy: tensor(0.7518, device='cuda:0')
Batch Loss: 277.0095291733742 Accuracy: tensor(0.7499, device='cuda:0')
Batch Loss: 351.9751906096935 Accuracy: tensor(0.7458, device='cuda:0')
Batch Loss: 421.41768875718117 Accuracy: tensor(0.7471, device='cuda:0')
Batch Loss: 491.3103261291981 Accuracy: tensor(0.7473, device='cuda:0')
Train Loss: 543.2810726463795 Accuracy: tensor(0.7458, device='cuda:0')


In [ ]:
losses, accuracies

([1119.791349887848,
  979.4726040363312,
  927.5939485430717,
  879.6240128874779,
  830.7548680901527,
  777.1825244426727,
  716.812304109335,
  658.5395728051662,
  599.2396767437458,
  543.2810726463795],
 [tensor(0.5065, device='cuda:0'),
  tensor(0.5504, device='cuda:0'),
  tensor(0.5695, device='cuda:0'),
  tensor(0.5885, device='cuda:0'),
  tensor(0.6079, device='cuda:0'),
  tensor(0.6292, device='cuda:0'),
  tensor(0.6613, device='cuda:0'),
  tensor(0.6858, device='cuda:0'),
  tensor(0.7199, device='cuda:0'),
  tensor(0.7458, device='cuda:0')])

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "naver_map_kcbert_epoch10.pt") 

In [ ]:
# 모델 불러오기
model.load_state_dict(torch.load("./naver_map_model.pt"))

# TEST

## Evaluation

In [ ]:
# 테스트 데이터셋 정확도 확인하기

model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.type(torch.LongTensor)
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/1435 [00:00<?, ?it/s]

Accuracy: tensor(0.6317, device='cuda:0')


In [ ]:
# 리뷰 텍스트 predicted label(0,1) 확인

def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=256
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# gradient 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
        
    logits = outputs[0] 
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits) 
    return result

## Test Result

In [ ]:
test_pd = pd.read_csv('test_set.csv')

In [ ]:
score = [] # label - score

total_len = len(test_pd)

for cnt, review in enumerate(test_pd['리뷰']): 
  try:
    pred = sentiment_classifier(review) 
    score.append(pred) 
  except:
    score.append('pass') 
  
  if cnt % 100 == 0:
    print(cnt, '개 문장 분류 완료')

0 개 문장 분류 완료


/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


100 개 문장 분류 완료
200 개 문장 분류 완료
300 개 문장 분류 완료
400 개 문장 분류 완료
500 개 문장 분류 완료
600 개 문장 분류 완료
700 개 문장 분류 완료
800 개 문장 분류 완료
900 개 문장 분류 완료
1000 개 문장 분류 완료
1100 개 문장 분류 완료
1200 개 문장 분류 완료
1300 개 문장 분류 완료
1400 개 문장 분류 완료
1500 개 문장 분류 완료
1600 개 문장 분류 완료
1700 개 문장 분류 완료
1800 개 문장 분류 완료
1900 개 문장 분류 완료
2000 개 문장 분류 완료
2100 개 문장 분류 완료
2200 개 문장 분류 완료
2300 개 문장 분류 완료
2400 개 문장 분류 완료
2500 개 문장 분류 완료
2600 개 문장 분류 완료
2700 개 문장 분류 완료
2800 개 문장 분류 완료
2900 개 문장 분류 완료
3000 개 문장 분류 완료
3100 개 문장 분류 완료
3200 개 문장 분류 완료
3300 개 문장 분류 완료
3400 개 문장 분류 완료
3500 개 문장 분류 완료
3600 개 문장 분류 완료
3700 개 문장 분류 완료
3800 개 문장 분류 완료
3900 개 문장 분류 완료
4000 개 문장 분류 완료
4100 개 문장 분류 완료
4200 개 문장 분류 완료
4300 개 문장 분류 완료
4400 개 문장 분류 완료
4500 개 문장 분류 완료
4600 개 문장 분류 완료
4700 개 문장 분류 완료
4800 개 문장 분류 완료
4900 개 문장 분류 완료
5000 개 문장 분류 완료
5100 개 문장 분류 완료
5200 개 문장 분류 완료
5300 개 문장 분류 완료
5400 개 문장 분류 완료
5500 개 문장 분류 완료
5600 개 문장 분류 완료
5700 개 문장 분류 완료
5800 개 문장 분류 완료
5900 개 문장 분류 완료
6000 개 문장 분류 완료
6100 개 문장 분류 완료
6200 개 문장 분류 완료
6300 개 문장 분류 완료
6

In [ ]:
score

[[{'label': 'LABEL_9', 'score': 0.8877827525138855}],
 [{'label': 'LABEL_9', 'score': 0.993765115737915}],
 [{'label': 'LABEL_6', 'score': 0.6236177682876587}],
 [{'label': 'LABEL_9', 'score': 0.9947482943534851}],
 [{'label': 'LABEL_5', 'score': 0.5408391356468201}],
 [{'label': 'LABEL_0', 'score': 0.6274548768997192}],
 [{'label': 'LABEL_9', 'score': 0.855877697467804}],
 [{'label': 'LABEL_9', 'score': 0.9691891074180603}],
 [{'label': 'LABEL_9', 'score': 0.9725343585014343}],
 [{'label': 'LABEL_9', 'score': 0.9790089726448059}],
 [{'label': 'LABEL_9', 'score': 0.9695204496383667}],
 [{'label': 'LABEL_9', 'score': 0.9902541041374207}],
 [{'label': 'LABEL_9', 'score': 0.320308655500412}],
 [{'label': 'LABEL_9', 'score': 0.970966637134552}],
 [{'label': 'LABEL_9', 'score': 0.6964117288589478}],
 [{'label': 'LABEL_9', 'score': 0.870610237121582}],
 [{'label': 'LABEL_9', 'score': 0.989767849445343}],
 [{'label': 'LABEL_9', 'score': 0.7620729804039001}],
 [{'label': 'LABEL_9', 'score': 0.

In [ ]:
test_pd['predicted'] = 'pass' 
score_list = [] # score 

for i in range(len(score)): 
    test_pd['predicted'][i] = int(score[i][0].get('label')[-1])
    score_list.append(float(score[i][0].get('score')))

<ipython-input-25-ce2ecb8c46c6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pd['predicted'][i] = int(score[i][0].get('label')[-1])


In [ ]:
test_pd

,Unnamed: 0,리뷰,grade,라벨링,predicted
0,0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,9,9
1,1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,9,9
2,2,좋아요,5.0,9,6
3,3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,9,9
4,4,귀엽다,3.0,5,5
...,...,...,...,...,...
27924,27924,생각보다 그냥 그래욤,3.5,6,5
27925,27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,9,9
27926,27926,친절해요,5.0,9,6
27927,27927,좋아요,5.0,9,6


In [ ]:
score_df = pd.DataFrame({'score' : score_list})
final_test_pd = pd.concat([test_pd, score_df], axis=1)
final_test_pd

,Unnamed: 0,리뷰,grade,라벨링,predicted,score
0,0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,9,9,0.887783
1,1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,9,9,0.993765
2,2,좋아요,5.0,9,6,0.623618
3,3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,9,9,0.994748
4,4,귀엽다,3.0,5,5,0.540839
...,...,...,...,...,...,...
27924,27924,생각보다 그냥 그래욤,3.5,6,5,0.534132
27925,27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,9,9,0.988582
27926,27926,친절해요,5.0,9,6,0.735431
27927,27927,좋아요,5.0,9,6,0.623618


In [ ]:
# 예측 패스한 문장 지우기
dropidx = final_test_pd[final_test_pd['predicted'] == 'pass'].index
final_test_pd.drop(dropidx, inplace=True)
final_test_pd

,Unnamed: 0,리뷰,grade,라벨링,predicted,score
0,0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,9,9,0.887783
1,1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,9,9,0.993765
2,2,좋아요,5.0,9,6,0.623618
3,3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,9,9,0.994748
4,4,귀엽다,3.0,5,5,0.540839
...,...,...,...,...,...,...
27924,27924,생각보다 그냥 그래욤,3.5,6,5,0.534132
27925,27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,9,9,0.988582
27926,27926,친절해요,5.0,9,6,0.735431
27927,27927,좋아요,5.0,9,6,0.623618


In [ ]:
# 결과 저장
final_test_pd.to_csv("kcbert_result.csv", encoding = 'utf-8-sig') 

In [ ]:
# 전체 결과 정확도 

# from sklearn.metrics import accuracy_score
# print(accuracy_score(final_test_pd['라벨링'], final_test_pd['predicted']))

sum(final_test_pd['라벨링'] == final_test_pd['predicted']) / len(final_test_pd)

0.5626051774141573